In [ ]:
from robovast.common.analysis import read_output_files, read_output_csv

DATA_DIR = ''

# Read all CSV files into a combined dataframe
combined_df = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "out.csv", skiprows=1))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Sequence, Tuple

def _pivot_by_test(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return pivot with index=time, columns=test, values=population.
    Each column corresponds to a test (i.e., a single run).
    """
    pivot = df.pivot(index='time', columns='test', values='population')
    pivot = pivot.sort_index()
    # If columns not unique, make unique by suffixing counters (rare)
    cols = list(pivot.columns)
    seen = {}
    new_cols = []
    for c in cols:
        if c not in seen:
            seen[c] = 1
            new_cols.append(c)
        else:
            seen[c] += 1
            new_cols.append(f"{c}__{seen[c]}")
    pivot.columns = new_cols
    return pivot

def _ecdf(arr: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    arr = np.sort(arr)
    y = np.arange(1, len(arr)+1) / len(arr)
    return arr, y

def tests_summary_dashboard(
    combined_df: pd.DataFrame,
    sample_for_spaghetti: int = 100,
    q_lo: float = 0.10,
    q_hi: float = 0.90,
    heatmap_bins: int = 80,
    box_times: Sequence[int] = (0, 25, 50, 75, 100),
    threshold_for_ecdf: Optional[float] = None,
    figsize: Tuple[int,int] = (14, 12),
    save_path: Optional[str] = None
) -> pd.DataFrame:
    """
    Create a multi-panel summary dashboard for all 'test' runs found in combined_df.
    Returns a summary DataFrame per test.
    """
    df = combined_df.copy()
    pivot = _pivot_by_test(df)
    if pivot.shape[1] == 0:
        raise ValueError("No tests found in combined_df.")
    n_tests = pivot.shape[1]
    times = pivot.index.values
    tmin, tmax = times.min(), times.max()

    # mean + percentile
    mean_traj = pivot.mean(axis=1)
    lo = pivot.quantile(q_lo, axis=1)
    hi = pivot.quantile(q_hi, axis=1)

    # heatmap matrix
    flat = pivot.values.flatten()
    flat = flat[np.isfinite(flat)]
    if flat.size == 0:
        raise ValueError("No finite population values.")
    vmin, vmax = flat.min(), flat.max()
    bin_edges = np.linspace(vmin, vmax, heatmap_bins + 1)
    hist_matrix = []
    for t in times:
        arr = pivot.loc[t].dropna().values
        if arr.size == 0:
            hist = np.zeros(heatmap_bins, dtype=float)
        else:
            hist, _ = np.histogram(arr, bins=bin_edges)
        hist_matrix.append(hist)
    hist_matrix = np.array(hist_matrix).T  # (bins, times)
    extent = (tmin, tmax, bin_edges[0], bin_edges[-1])

    # per-test summary
    final_time = times.max()
    rows = []
    for col in pivot.columns:
        ser = pivot[col].dropna()
        final_val = ser.loc[final_time] if final_time in ser.index else np.nan
        max_val = ser.max() if ser.size>0 else np.nan
        mean_val = ser.mean() if ser.size>0 else np.nan
        half_max = 0.5 * max_val if not np.isnan(max_val) else np.nan
        t_to_half = (int(ser[ser >= half_max].index[0]) if ser.size>0 and (ser >= half_max).any() else np.nan)
        rows.append({
            'test': col,
            'final_population': final_val,
            'max_population': max_val,
            'mean_population': mean_val,
            'time_to_half_max': t_to_half
        })
    summary_df = pd.DataFrame(rows).set_index('test')

    # plotting
    fig = plt.figure(constrained_layout=True, figsize=figsize)
    gs = fig.add_gridspec(3, 2)

    ax_spag = fig.add_subplot(gs[0, 0])
    ax_mean = fig.add_subplot(gs[0, 1])
    ax_heat = fig.add_subplot(gs[1, 0])
    ax_box  = fig.add_subplot(gs[1, 1])
    ax_hist = fig.add_subplot(gs[2, 0])
    ax_ecdf = fig.add_subplot(gs[2, 1])

    # Spaghetti (sample for readability)
    if n_tests > sample_for_spaghetti:
        rng = np.random.default_rng(0)
        chosen = rng.choice(pivot.columns, size=sample_for_spaghetti, replace=False)
        note = f"(showing {sample_for_spaghetti}/{n_tests} tests)"
    else:
        chosen = pivot.columns
        note = f"(showing {n_tests} tests)"

    for c in chosen:
        ax_spag.plot(times, pivot[c], alpha=0.3, linewidth=0.9)
    ax_spag.set_title(f"Spaghetti plot {note}")
    ax_spag.set_xlabel("time"); ax_spag.set_ylabel("population")
    ax_spag.grid(alpha=0.25)

    # Mean + ribbon
    ax_mean.plot(times, mean_traj, linewidth=2, label='mean')
    ax_mean.fill_between(times, lo, hi, alpha=0.25, label=f'{int(100*q_lo)}-{int(100*q_hi)} pctile')
    ax_mean.set_title("Mean trajectory with percentile ribbon")
    ax_mean.set_xlabel("time"); ax_mean.set_ylabel("population")
    ax_mean.legend()
    ax_mean.grid(alpha=0.25)

    # Heatmap
    im = ax_heat.imshow(hist_matrix, aspect='auto', origin='lower', extent=extent)
    fig.colorbar(im, ax=ax_heat, label='count')
    ax_heat.set_title("Ensemble heatmap (density over time)")
    ax_heat.set_xlabel("time"); ax_heat.set_ylabel("population")

    # Boxplots at selected times (use available times)
    avail_times = [t for t in box_times if t in pivot.index]
    if len(avail_times) == 0:
        # fallback: pick 5 evenly spaced time indices
        idx = np.linspace(0, len(times)-1, 5).astype(int)
        avail_times = list(np.unique(times[idx]))
    data_box = [pivot.loc[t].dropna().values for t in avail_times]
    ax_box.boxplot(data_box, tick_labels=[str(t) for t in avail_times], showfliers=False)
    ax_box.set_title("Boxplots at selected times")
    ax_box.set_xlabel("time"); ax_box.set_ylabel("population")
    ax_box.grid(alpha=0.25)

    # Final-time histogram
    finals = pivot.loc[final_time].dropna().values
    ax_hist.hist(finals, bins=20, alpha=0.9)
    ax_hist.set_title(f"Final-time histogram (time={final_time})")
    ax_hist.set_xlabel("population"); ax_hist.set_ylabel("count")
    ax_hist.grid(alpha=0.25)

    # ECDF of time-to-threshold (if requested)
    if threshold_for_ecdf is not None:
        times_to_thresh = []
        for c in pivot.columns:
            ser = pivot[c].dropna()
            reached = ser[ser >= threshold_for_ecdf]
            if reached.size == 0:
                continue
            times_to_thresh.append(int(reached.index[0]))
        if len(times_to_thresh) == 0:
            ax_ecdf.text(0.5, 0.5, "No tests reached threshold", ha='center', va='center')
            ax_ecdf.set_title(f"ECDF of time-to-threshold={threshold_for_ecdf}")
        else:
            x, y = _ecdf(np.array(times_to_thresh))
            ax_ecdf.step(x, y, where='post')
            ax_ecdf.set_xlim(min(x)*0.98, max(x)*1.02)
            ax_ecdf.set_ylim(0, 1.02)
            ax_ecdf.set_xlabel("time to reach threshold")
            ax_ecdf.set_ylabel("ECDF")
            ax_ecdf.set_title(f"ECDF of time-to-threshold={threshold_for_ecdf}")
            ax_ecdf.grid(alpha=0.25)
    else:
        ax_ecdf.text(0.5, 0.5, "ECDF omitted\n(no threshold provided)", ha='center', va='center')
        ax_ecdf.set_title("ECDF (omitted)")
    # Add categories (skip 'variant' and 'test') to suptitle
    cat_cols = [c for c in combined_df.columns if str(combined_df[c].dtype) == 'category' and c not in ('variant', 'test')]
    cat_vals = [f"{col}={combined_df[col].iloc[0]}" for col in cat_cols]
    cat_str = " | ".join(cat_vals) or ""
    plt.suptitle(f"Tests summary dashboard | tests={n_tests} | {cat_str}", fontsize=14)
    if save_path:
        fig.savefig(save_path, dpi=200, bbox_inches='tight')
        print(f"Saved dashboard to: {save_path}")
    plt.show()

    return summary_df

summary_df = tests_summary_dashboard(combined_df, threshold_for_ecdf=1000)